remap labels of `../../datasets/cross_validation_splits`

attenzione: no fa una copia ma modifica i file


In [1]:
from pathlib import Path
import yaml, shutil

# ===== Parametri utente =====
DATASET_ROOT = Path('../../datasets/cross_validation_splits')  # folder with fold_0/...
# ============================

assert DATASET_ROOT.exists(), f"Cartella {DATASET_ROOT} non trovata"
print('Dataset root:', DATASET_ROOT.resolve())


Dataset root: /home/andrea/work/AI-waste-detection/datasets/cross_validation_splits


In [2]:
# 7 classi finali
classes = ['glass', 'medical', 'metal', 'organic', 'paper', 'plastic', 'sharp-object']

# mappatura 42 -> 7
dict_classes = {
    'Aerosols': 2, 'Aluminum can': 2, 'Aluminum caps': 2, 'Cardboard': 4,
    'Cellulose': 4, 'Ceramic': 5, 'Combined plastic': 5,
    'Container for household chemicals': 1, 'Disposable tableware': 5,
    'Electronics': 2, 'Foil': 2, 'Furniture': 3, 'Glass bottle': 0,
    'Iron utensils': 2, 'Liquid': 5, 'Metal shavings': 2, 'Milk bottle': 4,
    'Organic': 3, 'Paper': 4, 'Paper bag': 4, 'Paper cups': 4,
    'Paper shavings': 4, 'Papier mache': 4, 'Plastic bag': 5,
    'Plastic bottle': 5, 'Plastic can': 5, 'Plastic canister': 5,
    'Plastic caps': 5, 'Plastic cup': 5, 'Plastic shaker': 5,
    'Plastic shavings': 5, 'Plastic toys': 5, 'Postal packaging': 5,
    'Printing industry': 4, 'Scrap metal': 2, 'Stretch film': 5,
    'Tetra pack': 4, 'Textile': 3, 'Tin': 2, 'Unknown plastic': 5,
    'Wood': 3, 'Zip plastic bag': 5,
}
orig_names = list(dict_classes.keys())
id_old_to_new = {i: dict_classes[name] for i, name in enumerate(orig_names)}


In [3]:
def remap_file(path: Path):
    """Sovrascrive un file YOLO cambiando l'ID di classe."""
    new_lines=[]
    with path.open() as f:
        for line in f:
            parts=line.strip().split()
            if len(parts)<5: continue
            old_id=int(parts[0])
            new_id=id_old_to_new.get(old_id)
            if new_id is None:
                print('Classe non mappata:', old_id, 'in', path)
                continue
            parts[0]=str(new_id)
            new_lines.append(' '.join(parts))
    path.write_text('\n'.join(new_lines)+'\n')


In [4]:
total=0
for fold in sorted(DATASET_ROOT.glob('fold_*')):
    print('\n📂', fold.name)
    for split in ['train','val']:
        lbl_dir=fold/split/'labels'
        if not lbl_dir.exists():
            print('  ⏩ Nessun', lbl_dir)
            continue
        for txt in lbl_dir.glob('*.txt'):
            remap_file(txt)
            total+=1
        print('  🔄', split, ':', len(list(lbl_dir.glob('*.txt'))),'file convertiti')
    # aggiorna data.yaml
    data_yaml=fold/'data.yaml'
    data_yaml.write_text(yaml.dump({
        'path': str(fold.relative_to(DATASET_ROOT.parent.parent)), # ../../datasets/...
        'train':'train/images',
        'val':'val/images',
        'nc':len(classes),
        'names':{i:n for i,n in enumerate(classes)},
    }, sort_keys=False))
    print('  ✅ data.yaml aggiornato')
print(f'\n🎉 Conversione terminata. File totali modificati: {total}')



📂 fold_0
  🔄 train : 9828 file convertiti
  🔄 val : 3276 file convertiti
  ✅ data.yaml aggiornato

📂 fold_1
  🔄 train : 9828 file convertiti
  🔄 val : 3276 file convertiti
  ✅ data.yaml aggiornato

📂 fold_2
  🔄 train : 9828 file convertiti
  🔄 val : 3276 file convertiti
  ✅ data.yaml aggiornato

📂 fold_3
  🔄 train : 9828 file convertiti
  🔄 val : 3276 file convertiti
  ✅ data.yaml aggiornato

🎉 Conversione terminata. File totali modificati: 52416
